# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [52]:
raw_data = "../WeatherPy/weather_data.csv"
city_data = pd.read_csv(raw_data)
city_df = pd.DataFrame(city_data)
city_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,Nanortalik,60.1432,-45.2371,33.69,69,100,1.30,GL,1621747565
1,George Town,5.4112,100.3354,91.35,66,20,8.05,MY,1621747389
2,George Town,5.4112,100.3354,91.35,66,20,8.05,MY,1621747389
3,Qaanaaq,77.4840,-69.3632,18.14,76,82,6.26,GL,1621747357
4,Hermanus,-34.4187,19.2345,59.90,50,32,7.65,ZA,1621747365


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [53]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [54]:
# Insert data f/ dataframe
locations = city_df[["Lat","Long"]]
humidity = city_df["Humidity"]

# Create heatmap gmap
humidity_heatmap = gmaps.figure(zoom_level=2,center=(0,0))

humidity_heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=3)

humidity_heatmap.add_layer(humidity_heat_layer)

humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
# Narrow down temp range to between 70 and 80 degrees, wind speed under 10 mph, and cloudiness less than 15%
hotel_df = city_df.loc[city_df["Max Temp"] > 70]
hotel_df = hotel_df.loc[city_df["Max Temp"] < 80]
hotel_df = hotel_df.loc[city_df["Wind Speed"] < 10]
hotel_df = hotel_df.loc[city_df["Cloudiness"] < 15]
hotel_df = hotel_df.dropna(how='any')
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime
12,Birao,10.2849,22.7882,78.55,57,4,4.52,CF,1621747566
39,Bethel,41.3712,-73.4140,74.91,82,1,0.00,US,1621747389
49,Abhā,18.2164,42.5053,75.02,43,0,2.98,SA,1621747572
159,Elizabeth City,36.2946,-76.2511,76.01,63,1,1.99,US,1621747591
167,La Providencia,28.7167,-111.5833,77.85,23,0,8.01,MX,1621747593
209,Atar,20.5169,-13.0499,78.01,40,0,4.03,MR,1621747447
267,São Filipe,14.8961,-24.4956,72.18,81,5,9.48,CV,1621747609
291,Westport,41.1415,-73.3579,77.76,80,1,7.36,US,1621747613
307,Mossamedes,-15.1961,12.1522,73.76,64,5,2.48,AO,1621747617
357,Cabedelo,-6.9811,-34.8339,72.27,77,1,9.22,BR,1621747508


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
# Add "Hotel Name" column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name
12,Birao,10.2849,22.7882,78.55,57,4,4.52,CF,1621747566,
39,Bethel,41.3712,-73.4140,74.91,82,1,0.00,US,1621747389,
49,Abhā,18.2164,42.5053,75.02,43,0,2.98,SA,1621747572,
159,Elizabeth City,36.2946,-76.2511,76.01,63,1,1.99,US,1621747591,
167,La Providencia,28.7167,-111.5833,77.85,23,0,8.01,MX,1621747593,
209,Atar,20.5169,-13.0499,78.01,40,0,4.03,MR,1621747447,
267,São Filipe,14.8961,-24.4956,72.18,81,5,9.48,CV,1621747609,
291,Westport,41.1415,-73.3579,77.76,80,1,7.36,US,1621747613,
307,Mossamedes,-15.1961,12.1522,73.76,64,5,2.48,AO,1621747617,
357,Cabedelo,-6.9811,-34.8339,72.27,77,1,9.22,BR,1621747508,


In [70]:
# Populate hotel column
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

row = 0
for city in hotel_df["City"]:
    params = {
        "location": f"{hotel_df.iloc[row,1]}, {hotel_df.iloc[row,2]}",
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key
    }
    try:
        hotel_response = requests.get(base_url, params=params).json()
        hotel_name = hotel_response['results'][0]['name']
        hotel_df.iloc[row, 9] = hotel_name
    except IndexError:
        hotel_df.iloc[row, 9] = "No Hotels"
    row += 1
    
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name
12,Birao,10.2849,22.7882,78.55,57,4,4.52,CF,1621747566,No Hotels
39,Bethel,41.3712,-73.4140,74.91,82,1,0.00,US,1621747389,Courtyard by Marriott Danbury
49,Abhā,18.2164,42.5053,75.02,43,0,2.98,SA,1621747572,Sarawat Park Hotel & Resort
159,Elizabeth City,36.2946,-76.2511,76.01,63,1,1.99,US,1621747591,Fairfield Inn & Suites by Marriott Elizabeth City
167,La Providencia,28.7167,-111.5833,77.85,23,0,8.01,MX,1621747593,No Hotels
209,Atar,20.5169,-13.0499,78.01,40,0,4.03,MR,1621747447,نزل البحر والصحراء
267,São Filipe,14.8961,-24.4956,72.18,81,5,9.48,CV,1621747609,Hotel Xaguate
291,Westport,41.1415,-73.3579,77.76,80,1,7.36,US,1621747613,The Inn at Longshore by OnTheMarc
307,Mossamedes,-15.1961,12.1522,73.76,64,5,2.48,AO,1621747617,iu Hotel Namibe
357,Cabedelo,-6.9811,-34.8339,72.27,77,1,9.22,BR,1621747508,Hotel Almagre


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [75]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity_heatmap.add_layer(marker_layer)

# Display figure
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))